In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import os
import torch
from PIL import Image

In [4]:
class ImageFolderWithLabels(Dataset):
    def __init__(self, root_dir, transform=None, half_size=False):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self._get_image_paths_and_labels(half_size)

    def _get_image_paths_and_labels(self, half_size=False):
      for subfolder in ["real", "fake"]:
            subfolder_path = os.path.join(self.root_dir, subfolder)
            image_list = os.listdir(subfolder_path)  # Get all filenames

            if half_size:
                image_list = image_list[::2]  # Select every alternate image

            for filename in image_list:
                if filename.endswith(".jpg"):
                    img_path = os.path.join(subfolder_path, filename)
                    self.image_paths.append(img_path)
                    self.labels.append(0 if subfolder == "real" else 1)  # Assign label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)

        return img, label

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet statistics
])

In [6]:
# Load data from each directory
train_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/celebDF_processed_split', 'Train'), transform=transform, half_size=True)
test_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/celebDF_processed_split', 'Test'), transform=transform, half_size=True)
val_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/celebDF_processed_split', 'val'), transform=transform, half_size=True)

In [7]:
print(len(train_data))
print(len(test_data))
print(len(val_data))

4723
590
592


In [8]:
BATCH_SIZE=32

In [9]:
# Create dataloaders for training, testing, and validation
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)  # Adjust batch size as needed
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
vit = torchvision.models.vit_b_16(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 142MB/s]


In [11]:
total_params = sum(p.numel() for p in vit.parameters())
total_params

86567656

In [12]:
import torch.optim as optim
optimizer = optim.Adam(vit.parameters(), lr=0.001)

In [13]:
# Define your loss function
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [14]:
# Define the device to run the model on (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
vit.to(device)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [16]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=8):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        # Training phase
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        # Calculate training loss and accuracy
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct / total

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = model(val_inputs)
                val_loss = criterion(val_outputs, val_labels)
                val_running_loss += val_loss.item() * val_inputs.size(0)
                _, val_predicted = val_outputs.max(1)
                val_total += val_labels.size(0)
                val_correct += val_predicted.eq(val_labels).sum().item()

        # Calculate validation loss and accuracy
        epoch_val_loss = val_running_loss / len(val_loader.dataset)
        epoch_val_acc = val_correct / val_total

        # Print epoch statistics
        print(f"Epoch {epoch + 1}/{num_epochs}",
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}",
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    print('Training complete')


num_epochs = 10


train_model(vit, train_loader, val_loader, criterion, optimizer, device, num_epochs)


Epoch 1/10 Train Loss: 0.8456, Train Acc: 0.5204 Val Loss: 0.6809, Val Acc: 0.5642
Epoch 2/10 Train Loss: 0.7022, Train Acc: 0.5539 Val Loss: 0.6788, Val Acc: 0.5709
Epoch 3/10 Train Loss: 0.6803, Train Acc: 0.5736 Val Loss: 0.7068, Val Acc: 0.5524
Epoch 4/10 Train Loss: 0.6791, Train Acc: 0.5797 Val Loss: 0.6557, Val Acc: 0.6334
Epoch 5/10 Train Loss: 0.6557, Train Acc: 0.6070 Val Loss: 0.6736, Val Acc: 0.5895
Epoch 6/10 Train Loss: 0.6516, Train Acc: 0.6193 Val Loss: 0.6322, Val Acc: 0.6284
Epoch 7/10 Train Loss: 0.6410, Train Acc: 0.6225 Val Loss: 0.6529, Val Acc: 0.6385
Epoch 8/10 Train Loss: 0.6163, Train Acc: 0.6517 Val Loss: 0.6115, Val Acc: 0.6689
Epoch 9/10 Train Loss: 0.6059, Train Acc: 0.6663 Val Loss: 0.6147, Val Acc: 0.6723
Epoch 10/10 Train Loss: 0.5997, Train Acc: 0.6727 Val Loss: 0.5892, Val Acc: 0.6976
Training complete


In [17]:
def test_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # No need to calculate gradients during testing
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss = running_loss / len(test_loader.dataset)
    test_acc = correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [18]:
test_model(vit, test_loader, criterion, device)

Test Loss: 0.5874, Test Accuracy: 0.6797


In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    confusion_mat = confusion_matrix(true_labels, predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(confusion_mat)

#Assuming you have already defined your test_loader and model
evaluate_model(vit, test_loader, device)



Accuracy: 0.6797
Precision: 0.6645
Recall: 0.7123
F1 Score: 0.6876
Confusion Matrix:
[[193 105]
 [ 84 208]]
